In [8]:
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torch
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from PIL import Image

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Operators:
    @staticmethod
    def PauliX():
        return torch.tensor([[0, 1], [1, 0]], dtype=torch.complex64, device=device)

    @staticmethod
    def PauliY():
        return torch.tensor([[0, -1j], [1j, 0]], dtype=torch.complex64, device=device)

    @staticmethod
    def PauliZ():
        return torch.tensor([[1, 0], [0, -1]], dtype=torch.complex64, device=device)

    @staticmethod
    def Rx(theta):
        return torch.cos(theta / 2) * torch.eye(2, dtype=torch.complex64, device=device) - \
               1j * torch.sin(theta / 2) * Operators.PauliX()

    @staticmethod
    def Ry(theta):
        return torch.cos(theta / 2) * torch.eye(2, dtype=torch.complex64, device=device) - \
               1j * torch.sin(theta / 2) * Operators.PauliY()

    @staticmethod
    def Rz(theta):
        return torch.cos(theta / 2) * torch.eye(2, dtype=torch.complex64, device=device) - \
               1j * torch.sin(theta / 2) * Operators.PauliZ()

    @staticmethod
    def CNOT():
        I = torch.eye(2, dtype=torch.complex64, device=device)
        zero_proj = torch.tensor([[1, 0], [0, 0]], dtype=torch.complex64, device=device)
        one_proj = torch.tensor([[0, 0], [0, 1]], dtype=torch.complex64, device=device)
        return torch.kron(zero_proj, I) + torch.kron(one_proj, Operators.PauliX())

class utils:
    def apply_one_site(site, state, op, L=32):
        """ 
        Applies a single-site operator to a quantum state in tensorspace.

        Args:
            site  (int):          The site index on which the operator acts
            state (torch.tensor): The state vector of the full system
            op    (torch.tensor): A 2x2 operator acting on a single qubit
            L     (int):          The total number of sites in the system

        Returns:
            torch.tensor: The resulting state after applying the operator
        
        """
        state = state.reshape(2**(site), 2, 2**(L-site-1))
        state = torch.tensordot(op, state, dims=([1], [1]))
        state = state.permute(1, 0, 2).contiguous()
        return state.reshape(-1,)

    def apply_two_site(sites, state, op, L=32):
        """
        Applies a (nearest neighbor) two-site operator to a quantum state tensor.

        Args:
            sites (tuple): A tuple (i, j) indicating the two site indices the operator acts on, where i < j.
            state (torch.Tensor): The state vector of the full system (length 2^L).
            op (torch.Tensor): A 4x4 operator acting on two qubits.
            L (int, optional): The total number of sites in the system. Default is 32.

        Returns:
            torch.Tensor: The resulting state vector after applying the operator at the specified sites.
        """
        state = state.reshape(2**sites[0], 2, 2**(sites[1]-sites[0]-1), 2, 2**(L-sites[1]-1))
        op = op.view(2, 2, 2, 2)
        state = torch.tensordot(op, state, dims=([2, 3], [1, 3]))
        state = state.permute(2, 0, 3, 1, 4).contiguous()
        return state.reshape(-1,)

    def measure_expectation(state, op, site, L=32):
        """ 
        Measures the expectation value of a single-site operator on a given quantum state.

        Args:
            state (torch.Tensor): The state vector of the full system (length 2^L).
            op (torch.Tensor): A 2x2 Hermitian operator representing the observable.
            site (int): The site at which the measurement is performed.
            L (int, optional): The total number of sites in the system. Default is 32.

        Returns:
            float: The real part of the expectation value of the operator at the specified site.
     
        """
        state = state.view(2**(site), 2, 2**(L-site-1))
        measured_state = torch.tensordot(op, state, dims=([1], [1]))
        expectation_value = torch.vdot(state.view(-1), measured_state.view(-1)).real
        return expectation_value

def layer(params, state, L=32):
    """ 
    Definition of a single fully entangling layer of a quantum circuit

    Each layer consists of:
     - Rotation Ry applied to every qubit with corresponding parameter 
       (Can be exchanged to every other rotation, always followed by the application)
     - A chain of CNOT gates applied between adjacent qubits. 

    Args:
     params (torch.tensor):  a 1D tensor of length L, each (trainable) entry corresponding to a Ry angle
     state  (torch.tensor):  The state of the full system
     
    Returns:
     torch.tensor: Quantum state after applying the layer operations
    
    """
    for i in range(L):
        Ry_gate = Operators.Ry(params[i])
        state = utils.apply_one_site(i, state, Ry_gate, L)
    for i in range(L-1):
        state = utils.apply_two_site([i, i+1], state, Operators.CNOT(), L)
    return state

def circuit(params, state, L=32):
    n_layers = params.shape[0]
    for i in range(n_layers):
        state = layer(params[i], state, L=L)
    return utils.measure_expectation(state, Operators.PauliZ(), site=7, L=L)

# Load and preprocess MNIST dataset
from sklearn.datasets import fetch_openml
from PIL import Image

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
images, labels = mnist.data, mnist.target.astype(int)

indices_2 = np.where(labels == 2)[0][:2000]
indices_3 = np.where(labels == 3)[0][:2000]
selected_indices = np.concatenate((indices_2, indices_3))
selected_images = images[selected_indices]
selected_labels = labels[selected_indices]

selected_labels = np.where(selected_labels == 2, -1, 1)

def preprocess_image(image):
    image = image.reshape(28, 28)
    image_resized = Image.fromarray(np.uint8(image))
    image_resized = image_resized.resize((16, 16), Image.Resampling.LANCZOS)
    image_resized = np.array(image_resized).reshape(-1,)
    return image_resized / np.linalg.norm(image_resized)

selected_images = np.array([preprocess_image(img) for img in selected_images])

X_data = torch.tensor(selected_images, dtype=torch.float32, device=device)
Y_data = torch.tensor(selected_labels, dtype=torch.float32, device=device)

train_size = int(0.8 * len(X_data))
X_train, X_test = X_data[:train_size], X_data[train_size:]
Y_train, Y_test = Y_data[:train_size], Y_data[train_size:]

batch_size = 50
train_loader = torch.utils.data.DataLoader(list(zip(X_train, Y_train)), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(list(zip(X_test, Y_test)), batch_size=batch_size, shuffle=False)

print("Data preprocessing complete! Ready for training.")

class QuantumCircuitModel(nn.Module):
    def __init__(self, n_layers=32, n_qubits=8):
        super().__init__()
        self.n_layers = n_layers
        self.n_qubits = n_qubits
        self.params = nn.Parameter(torch.rand(n_layers, n_qubits) * torch.pi*0.01)  


    def forward(self, x):
        batch_size, input_dim = x.shape  
        assert input_dim == 2**self.n_qubits, f"Expected input_dim={2**self.n_qubits}, but got {input_dim}"
    

        state = x.to(dtype=torch.complex64,device=x.device) 

        results = torch.vmap(lambda s: circuit(self.params, s, L=self.n_qubits))(state)
    
        return results 


model = QuantumCircuitModel().to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

def convert_labels(y):
    return (y + 1) / 2  

epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct_train, total_train = 0, 0 

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = convert_labels(labels)  

        optimizer.zero_grad()
        outputs = model(images)  

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predictions = torch.sigmoid(outputs) > 0.5
        correct_train += (predictions == labels).sum().item()
        total_train += labels.size(0)
    
    train_accuracy = correct_train / total_train
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(train_loader):.4f},Train Accuracy: {train_accuracy * 100:.2f}%")

model.eval()
correct, total = 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        labels = convert_labels(labels)  

        outputs = model(images)
        predictions = torch.sigmoid(outputs) > 0.5  
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Data preprocessing complete! Ready for training.
Epoch [1/5], Loss: 0.6005,Train Accuracy: 87.53%
Epoch [2/5], Loss: 0.5622,Train Accuracy: 93.94%
Epoch [3/5], Loss: 0.5537,Train Accuracy: 94.19%
Epoch [4/5], Loss: 0.5498,Train Accuracy: 93.56%
Epoch [5/5], Loss: 0.5464,Train Accuracy: 94.03%
Test Accuracy: 82.00%
